***GENERATED CODE FOR bostonprednolsqls PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***SELECTED MODEL PREDICTION FUNCTION***

In [ ]:
import sklearn.metrics
import mlflow.sklearn
import pyspark
import mlflow
import sklearn
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


class ModelPredictionClass:

    def __init__(self, sparkDf, runID, label, mType, model_path, subtype):
        self.sparkDf = sparkDf
        self.runID = runID
        self.y = label
        self.yTest = []
        self.yPredicted = []
        self.mType = mType
        self.model_path = model_path
        self.subtype = subtype

    def getmetricsRegression(self):
        scoreMetrics = {}
        scoreMetrics["R2"] = round(
            sklearn.metrics.r2_score(self.yTest, self.yPredicted), 1)
        scoreMetrics["Mean_Squared_Error"] = round(
            sklearn.metrics.mean_squared_error(self.yTest, self.yPredicted), 1)

        return scoreMetrics

    def getmetricsClassification(self):
        scoreMetrics = {}
        self.yPredicted = [int(x) for x in self.yPredicted]
        self.yTest = [int(x) for x in self.yTest]
        scoreMetrics["Accuracy"] = round(
            (100 * sklearn.metrics.accuracy_score(y_true=self.yTest, y_pred=self.yPredicted)), 1)
        scoreMetrics["F1"] = round(
            (100 * sklearn.metrics.f1_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")), 1)
        scoreMetrics["Precision"] = round((
            100 * sklearn.metrics.precision_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")),
            1)
        scoreMetrics["Recall"] = round((
            100 * sklearn.metrics.recall_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")), 1)
        return scoreMetrics

    def getPrediction(self):
        scoreMetrics = {}
        probabilities = {}
        self.sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        self.sparkDf.toPandas()
        data = self.sparkDf.toPandas()
        complete_model_path = "runs:/" + self.runID + "/" + self.model_path
        self.yTest = list(data[self.y])
        data = data.drop(self.y, axis=1).values
        # model get
        sklearn_model = mlflow.sklearn.load_model(
            model_uri=complete_model_path)
        self.yPredicted = sklearn_model.predict(data).tolist()
        if "classification" == self.mType:
            self.yPredicted = [int(i) for i in self.yPredicted]
            scoreMetrics['result'] = self.getmetricsClassification()

            if self.subtype == "naivebayes" or self.subtype == "multinomialNB" or self.subtype == "GaussianNB" or self.subtype == "ComplementNB" or self.subtype == "BernoulliNB" or self.subtype == "CategoricalNB":
                classes = sklearn_model.classes_.tolist()
                scoreMetrics['classes'] = classes
                for i in range(len(classes)):
                    yproba = sklearn_model.predict_proba(data)[:, i].tolist()
                    my_list_rounded = list(
                        map(lambda x: round(x, ndigits=4), yproba))
                    col_name = "Probability_" + str(classes[i])
                    scoreMetrics[col_name] = my_list_rounded
                    probabilities[col_name] = my_list_rounded
                scoreMetrics['yTest'] = self.yTest
                scoreMetrics['yPredicted'] = self.yPredicted
                return scoreMetrics
            else:
                scoreMetrics['yTest'] = self.yTest
                scoreMetrics['yPredicted'] = self.yPredicted
                return scoreMetrics
        elif "regression" == self.mType:
            scoreMetrics['result'] = self.getmetricsRegression()
            scoreMetrics['yTest'] = self.yTest
            scoreMetrics['yPredicted'] = self.yPredicted
            return scoreMetrics

        elif "clustering" == self.mType:
            self.yPredicted = sklearn_model.predict(data).tolist()
            data['prediction'] = self.yPredicted
            return data


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run bostonprednolsqlsHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': '3.17.87.4', 'port': '1433', 'password': 'Passw0rd@123', 'database': 'testdb', 'user': 'SA', 'qtype': 'dbtable', 'query': 'BostonTest_NoL', 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'datacache': 0, 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICTING GIVEN DATA WITH SELECTED MODEL***

In [ ]:
#%run bostonprednolsqlsHooks.ipynb
try:
	#prediction-pipelinePreExecutionHook()

	sqlserver = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "CRIM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.66", "stddev": "8.64", "min": "0.00632", "max": "88.9762", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CRIM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ZN", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "11.5", "stddev": "23.43", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ZN"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INDUS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "11.13", "stddev": "6.9", "min": "0.46", "max": "27.74", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INDUS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CHAS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.07", "stddev": "0.26", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CHAS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NOX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.55", "stddev": "0.12", "min": "0.385", "max": "0.871", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NOX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.28", "stddev": "0.71", "min": "3.561", "max": "8.78", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AGE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "68.42", "stddev": "28.27", "min": "2.9", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AGE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DIS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.81", "stddev": "2.11", "min": "1.1296", "max": "12.1265", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DIS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RAD", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "9.64", "stddev": "8.72", "min": "1.0", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RAD"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TAX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "409.62", "stddev": "169.0", "min": "187.0", "max": "711.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TAX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PTRATIO", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "18.43", "stddev": "2.16", "min": "12.6", "max": "22.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PTRATIO"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "B", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "356.21", "stddev": "91.74", "min": "0.32", "max": "396.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "B"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LSTAT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "12.7", "stddev": "7.16", "min": "1.73", "max": "37.97", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LSTAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MEDV", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "22.57", "stddev": "9.24", "min": "5.0", "max": "50.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MEDV"}]}))

MPRe =  ModelPredictionClass(sqlserver,0e3a4c376de947eaa8f1935ecbede334, MEDV, regression, platformModelsTPOT_bostonhousingregression_20231102-053427_best_regressor_model, Automl):
responseMetrics = MPRe.getPrediction()
#prediction-pipelinePostExecutionHook(responseMetrics)


except Exception as ex: 
	logging.error(ex)
spark.stop()


***WRITING DATAFRAME***

In [ ]:
#%run bostonprednolsqlsHooks.ipynb
try:
	#sinkPreExecutionHook()

	sqlserver = RDBMSConnector.put(spark, "{'host': '3.17.87.4', 'port': '1433', 'password': 'Passw0rd@123', 'database': 'testdb', 'user': 'SA', 'optionalDB': '', 'table_option': 'create_new_table', 'table': 'BostonPred', 'new_table_name': 'BostonPred', 'writemode': '', 'is_header': 'Use Header Line', 'use_generatedColumns': False, 'generatedColumns': [], 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sinkPostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)
